In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [50]:
# Read the data
df = pd.read_csv("./CDA_raw_na.csv")

# Drop iso and keep country code for joining purpose
df.drop("iso", axis=1, inplace=True)
# Extract year info from column names "CDA.raw.YYYY"
df.columns = ["code", "country"] + [col.split(".")[-1] for col in df.columns[2:]]

selected_countries = [
    "United States of America",
    "China",
    "India",
    "Germany",
    "South Africa",
    "Australia",
    "Brazil",
]
filtered_df = df[df["country"].isin(selected_countries)]
filtered_df

,code,country,1999,2000,2001,2002,2003,2004,2005,2006,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
10,36,Australia,0.024391,0.026004,0.024047,0.023746,0.022140,0.018967,0.015285,0.015859,...,0.022055,0.019926,0.018043,0.011911,0.004440,-0.002348,-0.008829,-0.015156,-0.017659,-0.014269
26,76,Brazil,0.050379,0.050849,0.049526,0.042681,0.033882,0.026238,0.019763,0.015572,...,0.020151,0.026509,0.032619,0.038476,0.045319,0.045445,0.039429,0.033071,0.025264,0.010804
40,156,China,0.039081,0.031916,0.025075,0.022606,0.029114,0.042124,0.060659,0.081006,...,0.106040,0.098581,0.086566,0.075496,0.064731,0.053283,0.041920,0.031614,0.022560,0.014868
73,276,Germany,-0.032609,-0.021662,-0.012466,-0.011032,-0.010683,-0.011766,-0.014198,-0.009698,...,-0.025011,-0.024594,-0.022937,-0.017230,-0.015658,-0.013365,-0.008452,-0.006136,-0.006010,-0.021692
89,356,India,0.054609,0.052090,0.047498,0.042294,0.037049,0.034184,0.033508,0.036427,...,0.061657,0.065144,0.067467,0.065615,0.065725,0.063775,0.059530,0.054359,0.051362,0.049166
181,710,South Africa,0.021983,0.025384,0.023860,0.014334,0.011545,0.014620,0.016540,0.020467,...,0.031858,0.023559,0.009516,0.000213,-0.001003,-0.013999,-0.020481,-0.018434,-0.014359,-0.006917
209,840,United States of America,0.015146,0.016715,0.015046,0.013362,0.011827,0.010693,0.008992,0.006290,...,-0.005968,-0.011839,-0.020134,-0.025084,-0.024286,-0.021282,-0.021880,-0.018162,-0.010372,-0.011896


In [51]:
# Read the CSV file
weo_raw = pd.read_csv("./WEO_Data.csv")


filtered_weo = (
    weo_raw
    # Filter out records with unit of national currency. Keep records with unit of U.S. dollars for meaningful comparison.
    .query("Units == 'U.S. dollars'")
    .drop(
        ["ISO", "Country/Series-specific Notes", "Estimates Start After", "Units"],
        axis=1,
    )
    .rename(
        columns={
            "WEO Country Code": "code",
            "Country": "country",
            "Subject Descriptor": "type",
        }
    )
    .replace(
        {
            "Gross domestic product current prices": "GDP",
            "Gross domestic product per capita current prices": "GDPPC",
        }
    )
)
# 
filtered_weo.loc[filtered_weo["Scale"] == "Billions", filtered_weo.columns[4:]] *= 1000
weo = filtered_weo.drop("Scale", axis=1)

weo

,code,country,type,1995,1996,1997,1998,1999,2000,2001,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2,213,Argentina,GDP,288497.000,304282.000,327436.000,334244.000,316998.000,3.177590e+05,3.004210e+05,...,611471.00,563614.00,642464.00,556774.00,643861.00,524431.00,451815.00,389064.00,486702.00,632241.00
4,213,Argentina,GDPPC,8295.120,8645.460,9196.520,9283.180,8709.070,8.638540e+03,8.085360e+03,...,14488.83,13208.83,14895.32,12772.87,14618.33,11786.43,10054.02,8571.94,10616.95,13655.20
8,193,Australia,GDP,379071.000,424689.000,426424.000,381387.000,411727.000,3.999600e+05,3.777800e+05,...,1519020.00,1456600.00,1233100.00,1263520.00,1381620.00,1417030.00,1385260.00,1360690.00,1646390.00,1701890.00
10,193,Australia,GDPPC,20920.470,23168.960,23037.510,20388.910,21762.400,2.089540e+04,1.948680e+04,...,65200.37,61615.05,51412.25,51813.64,55797.38,56341.94,54266.75,53071.72,63896.30,65526.12
14,223,Brazil,GDP,770860.000,851146.000,883862.000,864307.000,599642.000,6.554540e+05,5.599820e+05,...,2471720.00,2456060.00,1800050.00,1796620.00,2063520.00,1916930.00,1873290.00,1476090.00,1648700.00,1924130.00
16,223,Brazil,GDPPC,4851.990,5204.570,5321.140,5123.550,3501.440,3.772050e+03,3.177940e+03,...,12358.34,12175.68,8846.48,8757.30,9978.09,9194.15,8914.17,6970.72,7754.61,8995.03
20,156,Canada,GDP,605941.000,630598.000,655010.000,634004.000,678417.000,7.446310e+05,7.389680e+05,...,1846600.00,1805750.00,1556510.00,1528000.00,1649270.00,1725300.00,1743730.00,1647600.00,2001490.00,2139840.00
22,156,Canada,GDPPC,20706.730,21325.690,21930.490,21046.560,22340.550,2.429672e+04,2.385971e+04,...,52708.61,51020.84,43626.47,42382.64,45191.99,46625.86,46449.96,43383.71,52387.81,55085.45
26,924,China,GDP,730996.000,860468.000,957991.000,1024170.000,1088350.000,1.205530e+06,1.333650e+06,...,9624930.00,10524240.00,11113510.00,11226900.00,12265330.00,13841810.00,14340600.00,14862560.00,17759310.00,18100040.00
28,924,China,GDPPC,603.526,703.060,774.910,820.903,865.236,9.511630e+02,1.044960e+03,...,7039.57,7645.88,8034.29,8063.45,8760.26,9848.95,10170.06,10525.00,12572.07,12813.77
